In [2]:
import pandas as pd
import numpy as np
import math
import time
import matplotlib.pyplot as plt

In [5]:
time.time()

1754419413.2627566

In [221]:
def func1(i):
    ### Total 0/1 network combinations
    Term1 = (2**i**2) 
    ### Subtract out the Combinations in which the diagonals of the matrix are 0 for
    ### for a 3x3 matrix, this would be the 2**6 remaining combinations of the matrix if the diagonals are all 0
    Term2 = (2**(i**2-i))
    return Term1-Term2

### Split a Number into a random set of combinations
def split_number(N, splits):
    while splits >= N:
        return('choose another number')        
    ### generate locations for split 
    split_locations = [0]
    for i in range(splits):
        val_last = split_locations[i]
        # print(val_last)
        x = np.random.uniform()
        if x > val_last:
            split_locations.append(x)
        else:
            while x <= val_last:
                x = np.random.uniform()
                # print(i, x)
                if x > val_last:
                    split_locations.append(x)
                    break
    split_locations[-1] = 1
    return split_locations

    
def compute_differences(list_nums):
    ### compute differences
    differences = [list_nums[i]-list_nums[i-1] for i in range(1, len(list_nums))]
    ### multiply by differences

    return differences


def compute_bucket_quantities(N, differences):
    # Integer_Quantities = [int(np.max([1., np.floor(N*k)])) for k in differences]
    Integer_Quantities = []
    for k in differences:
        val = int(np.max([1., np.floor(N*k)]))
        checksum = sum(Integer_Quantities)
        if checksum < N:
            Integer_Quantities.append(val)
        else:
            Integer_Quantities.append(0)
        # print(checksum)
    return Integer_Quantities



In [271]:
# def split_number(A):
#     def split(A):
#         A1 = int(np.floor(A * np.random.uniform()))
#         A2 = A - A1
#         return A1, A2
#     A1, A2 = split(A)
#     while A1 <= 1 or A2 <=1:
#         A1, A2 = split(A)
#     return A1, A2

def special_sum(N, N_Split):
    if N < 1:
       raise ValueError("N must be at least 1")
    else:
        total_combinations = 1
        for n in N_Split:
            if n == 0:
                pass
            else:
                sub_combinations = 0
                for i in range(1, n+1):
                    term = math.comb(n, i) * func1(i)
                    sub_combinations += term
                    # print(sub_combinations)
            total_combinations *= sub_combinations
    return total_combinations

In [273]:
# special_sum(20, [19, 1]), special_sum(20, [20])

(4697076207218035326616411655444793727537155489268804758303759062448357402686533358501768854107683218230110167,
 2582247415555462839138835735274223977381791974891691247556635401676867949567634842924888562848143179168405792001771308028)

In [283]:
# counter = 0
# for i in range(1000):
#     try:
#         split_number(7, 5)
#     except:
#         counter+=1
#         print(counter)

# N = 7
# Splits = 5
# N_Bootstrap = 100

def Generate_Special_Sum_DF(N, Splits, N_Bootstrap):
    t0=time.time()
    counter = 0
    Special_Sum_Meta = special_sum(N, [N])
    list_Sub = []
    for k in range(N_Bootstrap):
        split_props = split_number(N, Splits)
        # print(split_props)
        differences = compute_differences(split_props)
        # print(differences)
        N_Split = compute_bucket_quantities(N, differences)
        # print(N_Split)
        list_Sub.append(special_sum(N, N_Split))
        counter+=1
        if counter % 20 == 0:
            print('Counter: ', counter)
            print('Minutes: ', (time.time()-t0)/t0)
    # list_Meta = [Special_Sub_Meta for i in range(len(list_Sub))]
    df = pd.DataFrame({'Sub': list_Sub})
    df['Meta'] = Special_Sum_Meta
    return df


Counter:  20
Minutes:  4.329124886111892e-11
Counter:  40
Minutes:  4.746880959867996e-11
Counter:  60
Minutes:  7.20594913638757e-11
Counter:  80
Minutes:  7.44384763764564e-11
Counter:  100
Minutes:  7.560323591848937e-11
CPU times: user 134 ms, sys: 998 μs, total: 135 ms
Wall time: 134 ms


In [337]:
%%time
N = [20]
Splits = [10]
N_Bootstrap = [100]


for n in N:
    for s in Splits:
        for b in N_Bootstrap:  
            ### If the number is too large and generates an error, increase the number of splits
            df = Generate_Special_Sum_DF(n, s, b)
            df['Sub'] = df['Sub'].astype(float)
            
            # Calculate median and IQR (Q1 and Q3) of list_Sub
            sub_median = np.median(df['Sub'])
            sub_mean = np.mean(df['Sub'])
            sub_q0 = np.percentile(df['Sub'], 0)
            sub_q1 = np.percentile(df['Sub'], 25)
            sub_q3 = np.percentile(df['Sub'], 75)
            sub_q4 = np.percentile(df['Sub'], 100)
            # Calculate median and IQR (Q1 and Q3) of list_Sub
            sub_median = math.log10(np.median(df['Sub']))
            sub_mean = math.log10(np.mean(df['Sub']))
            sub_q0 = math.log10(np.percentile(df['Sub'], 0))
            sub_q1 = math.log10(np.percentile(df['Sub'], 25))
            sub_q3 = math.log10(np.percentile(df['Sub'], 75))
            sub_q4 = math.log10(np.percentile(df['Sub'], 100))
            
            # Create figure with two subplots
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
            
            # 1. Scatter plot of Meta vs Sub
            ax1.scatter(df['Meta'], df['Sub'], alpha=0.5)
            ax1.set_xlabel('Log10(Meta Network)')
            ax1.set_ylabel('Log10(Sub Networks)')
            ax1.set_title('Scatter Plot: Meta vs Sub: Size {}'.format(N))
            ax1.grid(True, linestyle='--', alpha=0.7)
            
            # 2. Histogram of list_Sub with median and IQR lines
            ax2.hist(df['Sub'].apply(lambda x: math.log10(x)), bins=33, edgecolor='black', alpha=0.7)
            ax2.axvline(sub_median, color='red', linestyle=':', label=f'Median: {math.log10(sub_median):.2f}')
            ax2.axvline(sub_mean, color='red', linestyle=':', label=f'Mean: {math.log10(sub_mean):.2f}', linewidth=3)
            ax2.axvline(sub_q0, color='black', linestyle=':', label=f'Q0 (0th): {math.log10(sub_q0):.2f}', linewidth=3)
            ax2.axvline(sub_q1, color='green', linestyle=':', label=f'Q1 (25th): {math.log10(sub_q1):.2f}', linewidth=3)
            ax2.axvline(sub_q3, color='green', linestyle=':', label=f'Q3 (75th): {math.log10(sub_q3):.2f}', linewidth=3)
            ax2.axvline(sub_q4, color='black', linestyle=':', label=f'Q4 (100th): {math.log10(sub_q4):.2f}')
            ax2.set_xlabel('Log10(Sub Networks)')
            ax2.set_ylabel('Frequency')
            ax2.set_title('Histogram of Sub Network Log Values with IQR: Size {}: Splits {}: Boots {}'.format(n, s, b))
            ax2.grid(True, linestyle='--', alpha=0.7)  # Retain grid lines
            ax2.legend()
            
            # Adjust layout to prevent overlap
            plt.tight_layout()
            plt.show()

Counter:  20
Minutes:  3.107214786380081e-10
Counter:  40
Minutes:  9.303906753200907e-09
Counter:  60
Minutes:  3.022279969087828e-08
Counter:  80
Minutes:  3.2900809775397544e-08
Counter:  100
Minutes:  6.572188036643499e-08


OverflowError: int too large to convert to float

In [317]:
list_Sub

[]

In [6]:
def summation_s_n(n):
    """
    Computes the sum of S(k) from k=1 to N, where S(k) = sum from i=1 to k of (k choose i) * 2^(i^2 - i).
    Args:
        n (int): Upper limit of the outer summation (N >= 1).
    Returns:
        int: The result of the summation.
    Raises:
        ValueError: If n is less than 1.
    """
    if n < 1:
        raise ValueError("N must be at least 1")
    total = 0
    for k in range(1, n + 1):
        # Compute S(k) = sum from i=1 to k of (k choose i) * 2^(i^2 - i)
        s_k = 0
        for i in range(1, k + 1):
            term = math.comb(k, i) * func1(i)#(2**(i*i) - 2**(i*i-i))
            s_k += term
        total += s_k
    return total

summation_s_n(5)

1074207

In [7]:
# A = summation_s_n(5)
# B =  summation_s_n(3) * summation_s_n(2)
# A, B

In [218]:
list_Meta = []
list_Sub = []

A = 100
def split_number(A):
    def split(A):
        A1 = int(np.floor(A * np.random.uniform()))
        A2 = A - A1
        return A1, A2
    A1, A2 = split(A)
    while A1 <= 1 or A2 <=1:
        A1, A2 = split(A)
    return A1, A2

# for i in range(1000):
#     A1, A2 = split_number(A)
#     N_Meta_Network = summation_s_n(A)
#     N_Sub_Networks =  summation_s_n(A1) * summation_s_n(A2)
#     list_Meta.append(math.log10(N_Meta_Network))
#     list_Sub.append(math.log10(N_Sub_Networks))

# Create DataFrame
df = pd.DataFrame({'Meta': list_Meta, 'Sub': list_Sub})

# Create DataFrame
df = pd.DataFrame({'Meta': list_Meta, 'Sub': list_Sub})

# Calculate median and IQR (Q1 and Q3) of list_Sub
sub_median = np.median(list_Sub)
sub_mean = np.mean(list_Sub)
sub_q0 = np.percentile(list_Sub, 0)
sub_q1 = np.percentile(list_Sub, 25)
sub_q3 = np.percentile(list_Sub, 75)
sub_q4 = np.percentile(list_Sub, 100)

# Create figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# 1. Scatter plot of Meta vs Sub
ax1.scatter(df['Meta'], df['Sub'], alpha=0.5)
ax1.set_xlabel('Log10(Meta Network)')
ax1.set_ylabel('Log10(Sub Networks)')
ax1.set_title('Scatter Plot: Meta vs Sub: Size {}'.format(A))
ax1.grid(True, linestyle='--', alpha=0.7)

# 2. Histogram of list_Sub with median and IQR lines
ax2.hist(list_Sub, bins=20, edgecolor='black', alpha=0.7)
ax2.axvline(sub_median, color='red', linestyle=':', label=f'Median: {sub_median:.2f}')
ax2.axvline(sub_mean, color='red', linestyle=':', label=f'Mean: {sub_median:.2f}', linewidth=3)
ax2.axvline(sub_q0, color='black', linestyle=':', label=f'Q1 (0th): {sub_q1:.2f}', linewidth=3)
ax2.axvline(sub_q1, color='green', linestyle=':', label=f'Q1 (25th): {sub_q1:.2f}', linewidth=3)
ax2.axvline(sub_q3, color='black', linestyle=':', label=f'Q3 (75th): {sub_q3:.2f}', linewidth=3)
ax2.axvline(sub_q4, color='black', linestyle=':', label=f'Q1 (100th): {sub_q1:.2f}')
ax2.set_xlabel('Log10(Sub Networks)')
ax2.set_ylabel('Frequency')
ax2.set_title('Histogram of Sub Network Log Values with IQR: Size {}'.format(A))
ax2.grid(True, linestyle='--', alpha=0.7)  # Retain grid lines
ax2.legend()

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

/home/alsuar/anaconda3/envs/deep_learning/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/alsuar/anaconda3/envs/deep_learning/lib/python3.13/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


IndexError: index -1 is out of bounds for axis 0 with size 0

In [36]:
t0 =  time.time()
list_time = []
for k in range(100, 2200, 100):
    t1 = time.time()
    val = summation_s_n(k)
    log = math.log10(val)
    t2 = time.time()
    time_elapsed = (t2-t0)/60
    print('Segment Time: ', k, ': ', time_elapsed)
    print('Total Time: ', k, ': ', (t2-t0)/60)
    # print(val)
    print('Appx, Num Digits: ', log)
    list_time.append(time_elapsed)
    print('\n')

pd.plot(pd.DataFrame(list_time))

Segment Time:  100 :  0.00048671563466389975
Total Time:  100 :  0.00048671563466389975
Appx, Num Digits:  3010.299956639812


Segment Time:  200 :  0.005491721630096436
Total Time:  200 :  0.005491721630096436
Appx, Num Digits:  12041.199826559248


Segment Time:  300 :  0.03584648768107097
Total Time:  300 :  0.03584648768107097
Appx, Num Digits:  27092.69960975831


Segment Time:  400 :  0.14534886678059897
Total Time:  400 :  0.14534886678059897
Appx, Num Digits:  48164.79930623699


Segment Time:  500 :  0.3877873420715332
Total Time:  500 :  0.3877873420715332
Appx, Num Digits:  75257.4989159953


Segment Time:  600 :  0.9041422724723815
Total Time:  600 :  0.9041422724723815
Appx, Num Digits:  108370.79843903323


Segment Time:  700 :  1.890860637029012
Total Time:  700 :  1.890860637029012
Appx, Num Digits:  147504.69787535077


Segment Time:  800 :  3.6390781958897906
Total Time:  800 :  3.6390781958897906
Appx, Num Digits:  192659.19722494797


Segment Time:  900 :  6.4946107

KeyboardInterrupt: 